In [1]:
import pandas as pd, datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('data/Pizza_Sale.csv')
df1 = pd.read_csv('data/Pizza_ingredients.csv')

In [3]:
# Step 1: Create a dictionary mapping from (pizza_ingredients, pizza_size) to pizza_name_id
temp_df_for_null = df.dropna(subset=['pizza_name_id']).groupby(
    ['pizza_ingredients', 'pizza_size']
)['pizza_name_id'].first().to_dict()

# Step 2: Use the dictionary to fill missing values
df['pizza_name_id'] = df.apply(
    lambda row: temp_df_for_null.get((row['pizza_ingredients'], row['pizza_size']))
    if pd.isna(row['pizza_name_id']) else row['pizza_name_id'],
    axis=1
)


In [4]:
# Step 1: Create a dictionary mapping from (pizza_ingredients, pizza_size) to pizza_name_id
temp_df_for_null = df.dropna(subset=['pizza_ingredients']).groupby(
    ['pizza_name_id']
)['pizza_ingredients'].first().to_dict()

# Step 2: Use the dictionary to fill missing values
df['pizza_ingredients'] = df.apply(
    lambda row: temp_df_for_null.get((row['pizza_name_id']))
    if pd.isna(row['pizza_ingredients']) else row['pizza_ingredients'],
    axis=1
)

In [5]:
# Step 1: Create a dictionary mapping from (pizza_ingredients, pizza_size) to pizza_name_id
temp_df_for_null = df.dropna(subset=['pizza_category']).groupby(
    ['pizza_name']
)['pizza_category'].first().to_dict()

# Step 2: Use the dictionary to fill missing values
df['pizza_category'] = df.apply(
    lambda row: temp_df_for_null.get((row['pizza_name']))
    if pd.isna(row['pizza_category']) else row['pizza_category'],
    axis=1
)

In [6]:
# Step 1: Create a dictionary mapping from (pizza_ingredients, pizza_size) to pizza_name_id
temp_df_for_null = df.dropna(subset=['pizza_name']).groupby(
    ['pizza_name_id']
)['pizza_name'].first().to_dict()

# # Step 2: Use the dictionary to fill missing values
df['pizza_name'] = df.apply(
    lambda row: temp_df_for_null.get((row['pizza_name_id']))
    if pd.isna(row['pizza_name']) else row['pizza_name'],
    axis=1
)

In [7]:
# Step 1: Create a dictionary mapping from (pizza_ingredients, pizza_size) to pizza_name_id
temp_df_for_null = df.dropna(subset=['total_price','unit_price','quantity']).groupby(
    ['pizza_name_id']
)['unit_price'].first().to_dict()

# # Step 2: Use the dictionary to fill missing values
df['total_price'] = df.apply(
    lambda row: temp_df_for_null.get((row['pizza_name_id']),row['unit_price']*int(row['quantity']))
    if pd.isna(row['total_price']) else row['total_price'],
    axis=1
    
)

In [8]:
# need to standardize the date format
df['order_date'] = df['order_date'].str.replace('-', '/')

In [9]:
df['order_date'] = pd.to_datetime(df['order_date'],format="%d/%m/%Y", errors='raise')

In [10]:
# Extract features
df['year'] = df['order_date'].dt.year
df['month'] = df['order_date'].dt.month
df['day'] = df['order_date'].dt.day
df['day_of_week'] = df['order_date'].dt.dayofweek
df['week_number'] = df['order_date'].dt.isocalendar().week
df['is_weekend'] = df['order_date'].dt.dayofweek >= 5

In [11]:
def get_season(date):
    if date.month in [12, 1, 2]:
        return 'Winter'
    elif date.month in [3, 4, 5]:
        return 'Spring'
    elif date.month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['season'] = df['order_date'].apply(get_season)


In [12]:
xgb_df = df[['pizza_name_id','order_date','month','day','day_of_week','week_number','is_weekend','quantity']]

In [13]:
xgb_df['is_weekend'] = xgb_df['is_weekend'].astype(int)

C:\Users\User\AppData\Local\Temp\ipykernel_11932\3048334688.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xgb_df['is_weekend'] = xgb_df['is_weekend'].astype(int)


In [14]:
xgb_df.head()

,pizza_name_id,order_date,month,day,day_of_week,week_number,is_weekend,quantity
0,hawaiian_m,2015-01-01,1,1,3,1,0,1
1,classic_dlx_m,2015-01-01,1,1,3,1,0,1
2,five_cheese_l,2015-01-01,1,1,3,1,0,1
3,ital_supr_l,2015-01-01,1,1,3,1,0,1
4,mexicana_m,2015-01-01,1,1,3,1,0,1


In [15]:
# Aggregate the data by 'order_date' and 'pizza_name_id'
aggregated_df = xgb_df.groupby(['order_date', 'pizza_name_id']).agg({
    'quantity': 'sum'
}).reset_index()

# Create separate DataFrames for each pizza ID and resample weekly
pizza_ids = aggregated_df['pizza_name_id'].unique()
pizza_dfs = {}

for pizza_id in pizza_ids:
    temp_df = aggregated_df[aggregated_df['pizza_name_id'] == pizza_id]
    temp_df.set_index('order_date', inplace=True)
    temp_df = temp_df.drop('pizza_name_id', axis=1)

    # Resample weekly and sum quantities
    weekly_resampled = temp_df['quantity'].resample('W').sum().reset_index()

    # Add week_number, month, and week_of_month features
    weekly_resampled['week_number'] = weekly_resampled['order_date'].dt.isocalendar().week
    weekly_resampled['month'] = weekly_resampled['order_date'].dt.month
    weekly_resampled['week_of_month'] = (weekly_resampled['order_date'].dt.day - 1) // 7 + 1
    weekly_resampled.drop('order_date',axis=1, inplace=True)
    pizza_dfs[pizza_id] = weekly_resampled
    print(f"DataFrame for {pizza_id}:\n{pizza_dfs[pizza_id]}\n")

# Example output for verification
for pizza_name, df in pizza_dfs.items():
    print(f"DataFrame for {pizza_name}:\n{df}\n")


DataFrame for bbq_ckn_l:
    quantity  week_number  month  week_of_month
0         18            1      1              1
1         13            2      1              2
2         25            3      1              3
3         15            4      1              4
4         29            5      2              1
5         18            6      2              2
6         27            7      2              3
7         16            8      2              4
8         18            9      3              1
9         25           10      3              2
10        18           11      3              3
11        20           12      3              4
12        21           13      3              5
13        19           14      4              1
14        23           15      4              2
15        22           16      4              3
16        19           17      4              4
17        19           18      5              1
18        18           19      5              2
19        21   

In [16]:
def dummy_df(s_date, e_date):
    # Generate a date range for 2016
    date_range = pd.date_range(start=s_date, end=e_date)

    # Create a DataFrame with the date range
    df_2016 = pd.DataFrame(date_range, columns=['order_date'])

    # Extract features from the date
    df_2016['week_number'] = df_2016['order_date'].dt.isocalendar().week
    df_2016['month'] = df_2016['order_date'].dt.month
    df_2016['week_of_month'] = (df_2016['order_date'].dt.day - 1) // 7 + 1

    # Resample the DataFrame weekly
    df_for_testing = df_2016.resample('W', on='order_date').agg({
        'week_number': 'first',
        'month': 'first',
        'week_of_month': 'first'
    }).reset_index()
    df_for_testing.drop('order_date', axis=1, inplace=True)
    # Display the resulting DataFrame
    return df_for_testing

In [17]:
# List of pizza IDs
pizza_ids = list(pizza_dfs.keys())
predicted_dfs = {}

# Train the XGBoost model on each pizza dataset and calculate MAPE on the test data
for pizza_id in pizza_ids:
    X = pizza_dfs[pizza_id].drop(['quantity'], axis=1)
    y = pizza_dfs[pizza_id]['quantity']

    # Create a DMatrix for XGBoost with the entire dataset
    dtrain = xgb.DMatrix(X, label=y)
    
    # Generate the test DataFrame
    df_for_testing = dummy_df('2016-01-01', '2016-12-31')
    dtest = xgb.DMatrix(df_for_testing)

    # Set parameters for the XGBoost model
    params = {
        'objective': 'reg:squarederror',
        'max_depth': 6,
        'eta': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8
    }

    # Train the model on the entire dataset
    model = xgb.train(params, dtrain, num_boost_round=100)

    # Predict on the test set
    test_predictions = model.predict(dtest)

    # Add predictions to the test DataFrame
    df_for_testing['prediction'] = test_predictions
    predicted_dfs[pizza_id] = df_for_testing.reset_index(drop=True)

    print(f'DONE {pizza_id}')


DONE bbq_ckn_l
DONE bbq_ckn_m
DONE bbq_ckn_s
DONE big_meat_s
DONE calabrese_m
DONE cali_ckn_l
DONE cali_ckn_m
DONE cali_ckn_s
DONE ckn_alfredo_m
DONE ckn_alfredo_s
DONE ckn_pesto_l
DONE ckn_pesto_m
DONE classic_dlx_l
DONE classic_dlx_m
DONE classic_dlx_s
DONE five_cheese_l
DONE four_cheese_l
DONE four_cheese_m
DONE green_garden_l
DONE green_garden_m
DONE green_garden_s
DONE hawaiian_l
DONE hawaiian_m
DONE hawaiian_s
DONE ital_cpcllo_l
DONE ital_cpcllo_m
DONE ital_supr_l
DONE ital_supr_m
DONE ital_supr_s
DONE ital_veggie_s
DONE mediterraneo_l
DONE mediterraneo_m
DONE mexicana_l
DONE mexicana_m
DONE mexicana_s
DONE napolitana_l
DONE napolitana_s
DONE pep_msh_pep_l
DONE pep_msh_pep_m
DONE pep_msh_pep_s
DONE pepperoni_l
DONE pepperoni_m
DONE pepperoni_s
DONE peppr_salami_l
DONE peppr_salami_m
DONE peppr_salami_s
DONE prsc_argla_l
DONE prsc_argla_m
DONE prsc_argla_s
DONE sicilian_l
DONE sicilian_m
DONE sicilian_s
DONE soppressata_l
DONE southw_ckn_l
DONE southw_ckn_m
DONE southw_ckn_s
DONE 

In [18]:

for pizza_id in pizza_ids:
    predicted_dfs[pizza_id]['prediction'] = predicted_dfs[pizza_id]['prediction'].round().astype(int)

   
    # Merge the predicted values into the actual values DataFrame
    merged_df = pd.merge(
        predicted_dfs[pizza_id][['week_number', 'prediction']],
        pizza_dfs[pizza_id][['week_number', 'quantity']],
        on='week_number',
        how='inner'
    )
    merged_df = merged_df.fillna(0)
    # Replace zero values with a small constant
    merged_df['quantity'] = merged_df['quantity'].replace(0, 1)
    merged_df['prediction'] = merged_df['prediction'].replace(0, 1)

    # Calculate MAPE using sklearn's mean_absolute_percentage_error
    mape = mean_absolute_percentage_error(merged_df['quantity'],merged_df['prediction'])
    print(f'MAPE for {pizza_id} (sklearn calculation): {mape * 100:.2f}%')


MAPE for bbq_ckn_l (sklearn calculation): 2.54%
MAPE for bbq_ckn_m (sklearn calculation): 2.53%
MAPE for bbq_ckn_s (sklearn calculation): 4.11%
MAPE for big_meat_s (sklearn calculation): 1.89%
MAPE for calabrese_m (sklearn calculation): 3.46%
MAPE for cali_ckn_l (sklearn calculation): 5.20%
MAPE for cali_ckn_m (sklearn calculation): 1.96%
MAPE for cali_ckn_s (sklearn calculation): 3.77%
MAPE for ckn_alfredo_m (sklearn calculation): 3.79%
MAPE for ckn_alfredo_s (sklearn calculation): 1.96%
MAPE for ckn_pesto_l (sklearn calculation): 1.91%
MAPE for ckn_pesto_m (sklearn calculation): 3.49%
MAPE for classic_dlx_l (sklearn calculation): 2.49%
MAPE for classic_dlx_m (sklearn calculation): 3.03%
MAPE for classic_dlx_s (sklearn calculation): 2.78%
MAPE for five_cheese_l (sklearn calculation): 2.03%
MAPE for four_cheese_l (sklearn calculation): 3.05%
MAPE for four_cheese_m (sklearn calculation): 4.98%
MAPE for green_garden_l (sklearn calculation): 6.64%
MAPE for green_garden_m (sklearn calculat

Our model performace metrica are very good so we can proceed furthur for building automated predicting system

In [21]:
import json


# Convert DataFrames to dictionaries
predicted_dfs_dict = {pizza_id: df.to_dict(orient='list') for pizza_id, df in predicted_dfs.items()}

# Specify the file path
file_path = 'predicted_dfs.json'

# Save the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(predicted_dfs_dict, json_file, indent=4)

print("Dictionary has been successfully saved to a JSON file.")


Dictionary has been successfully saved to a JSON file.


In [68]:
file_path = 'predicted_dfs.json'

# Open and read the JSON file
with open(file_path, 'r') as json_file:
    data = json.load(json_file)

# Now 'data' is a dictionary containing the JSON data
print(data)


{'bbq_ckn_l': {'week_number': [53, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52], 'month': [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12], 'week_of_month': [1, 1, 2, 3, 4, 1, 2, 3, 4, 5, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 5, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 5, 1, 2, 3, 4, 1, 2, 3, 4, 5, 1, 2, 3, 4, 1, 2, 3, 4], 'prediction': [7, 18, 13, 25, 16, 28, 18, 27, 16, 18, 21, 21, 20, 20, 19, 23, 22, 20, 19, 18, 21, 25, 19, 12, 13, 19, 21, 25, 18, 17, 15, 21, 16, 15, 16, 20, 19, 18, 21, 17, 23, 14, 15, 12, 16, 20, 19, 20, 21, 19, 19, 23, 15]}, 'bbq_ckn_m': {'week_number': [53, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,

In [69]:
predicted_dfs = {}

# Load the dictionary from the JSON file

for pizza_id, data_dict in data.items():
    # Convert the lists to Series
    week_numbers = pd.Series(data_dict['week_number'])
    predictions = pd.Series(data_dict['prediction'])
    # Create a DataFrame from the Series
    df = pd.DataFrame({'week_number': week_numbers, 'prediction': predictions})
    predicted_dfs[pizza_id] = df
    print(f'DataFrame for {pizza_id}:\n{predicted_dfs[pizza_id]}\n')

DataFrame for bbq_ckn_l:
    week_number  prediction
0            53           7
1             1          18
2             2          13
3             3          25
4             4          16
5             5          28
6             6          18
7             7          27
8             8          16
9             9          18
10           10          21
11           11          21
12           12          20
13           13          20
14           14          19
15           15          23
16           16          22
17           17          20
18           18          19
19           19          18
20           20          21
21           21          25
22           22          19
23           23          12
24           24          13
25           25          19
26           26          21
27           27          25
28           28          18
29           29          17
30           30          15
31           31          21
32           32          16
33           33        

In [30]:
df1.head()

,pizza_name_id,pizza_name,pizza_ingredients,Items_Qty_In_Grams
0,bbq_ckn_l,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,bbq_ckn_l,The Barbecue Chicken Pizza,Red Peppers,15.0
2,bbq_ckn_l,The Barbecue Chicken Pizza,Green Peppers,20.0
3,bbq_ckn_l,The Barbecue Chicken Pizza,Tomatoes,30.0
4,bbq_ckn_l,The Barbecue Chicken Pizza,Red Onions,60.0


In [31]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   pizza_name_id       518 non-null    object 
 1   pizza_name          518 non-null    object 
 2   pizza_ingredients   518 non-null    object 
 3   Items_Qty_In_Grams  514 non-null    float64
dtypes: float64(1), object(3)
memory usage: 16.3+ KB


In [33]:
# Filter rows with any NaN values
df_with_na = df1[df1.isna().any(axis=1)]

print("DataFrame with rows containing NaN values:")
df_with_na


DataFrame with rows containing NaN values:


,pizza_name_id,pizza_name,pizza_ingredients,Items_Qty_In_Grams
24,brie_carre_s,The Brie Carre Pizza,Caramelized Onions,NaN
149,hawaiian_l,The Hawaiian Pizza,Sliced Ham,NaN
152,hawaiian_m,The Hawaiian Pizza,Sliced Ham,NaN
155,hawaiian_s,The Hawaiian Pizza,Sliced Ham,NaN


In [85]:
df1 = pd.read_csv('data/Pizza_ingredients.csv')
df1 = df1.dropna(subset='Items_Qty_In_Grams')

In [125]:
week = 1
predicted_quantities = {}

# Fetch the predicted quantity for the given week number
for pizza_id, df in predicted_dfs.items():
    predicted_quantity = int(df.loc[df['week_number'] == week, 'prediction'].values)
    predicted_quantities[pizza_id] = predicted_quantity
    print(f'Week {week} approximated selling QTY of {pizza_id} is: {predicted_quantity}')


Week 1 approximated selling QTY of bbq_ckn_l is: 18
Week 1 approximated selling QTY of bbq_ckn_m is: 15
Week 1 approximated selling QTY of bbq_ckn_s is: 5
Week 1 approximated selling QTY of big_meat_s is: 21
Week 1 approximated selling QTY of calabrese_m is: 3
Week 1 approximated selling QTY of cali_ckn_l is: 14
Week 1 approximated selling QTY of cali_ckn_m is: 8
Week 1 approximated selling QTY of cali_ckn_s is: 4
Week 1 approximated selling QTY of ckn_alfredo_m is: 8
Week 1 approximated selling QTY of ckn_alfredo_s is: 2
Week 1 approximated selling QTY of ckn_pesto_l is: 6
Week 1 approximated selling QTY of ckn_pesto_m is: 2
Week 1 approximated selling QTY of classic_dlx_l is: 7
Week 1 approximated selling QTY of classic_dlx_m is: 12
Week 1 approximated selling QTY of classic_dlx_s is: 6
Week 1 approximated selling QTY of five_cheese_l is: 21
Week 1 approximated selling QTY of four_cheese_l is: 12
Week 1 approximated selling QTY of four_cheese_m is: 8
Week 1 approximated selling QTY o

In [ ]:
def calculate_total_ingredients(predicted_dfs, df1, week):
   
    predicted_quantities = {}

    # Fetch the predicted quantity for the given week number
    for pizza_id, df in predicted_dfs.items():
        predicted_quantity = int(df.loc[df['week_number'] == week, 'prediction'].values)
        predicted_quantities[pizza_id] = predicted_quantity

    # Initialize an empty dictionary to hold the results
    temp_df1 = {}

    # Iterate over each pizza_id and its corresponding DataFrame in predicted_quantities
    for pizza_id, predicted_qty in predicted_quantities.items():
        # Filter df1 for the current pizza_id
        temp_df2 = df1[df1['pizza_name_id'] == pizza_id].copy()
        
        # Assign the predicted quantity to the DataFrame
        temp_df2['predicted_quantity'] = predicted_qty
        
        # Calculate the total quantity in grams
        temp_df2['total_qty_ingrams'] = temp_df2['predicted_quantity'] * temp_df2['Items_Qty_In_Grams']
        
        # Store the updated DataFrame in the dictionary
        temp_df1[pizza_id] = temp_df2

    # Merge all DataFrames in temp_df1 into a single DataFrame
    merged_df = pd.concat(temp_df1.values(), ignore_index=True)

    # Group by 'pizza_ingredients' and sum 'total_qty_ingrams'
    grouped_df = merged_df.groupby('pizza_ingredients')['total_qty_ingrams'].sum().reset_index()

    print("Grouped DataFrame with Total Ingredients Quantity:")
    return grouped_df



In [124]:
# Prompt the user to enter the week number
week = int(input("Enter the week number for order quantity: "))

# Call the function with the given inputs
grouped_df = calculate_total_ingredients(predicted_dfs, df1, week)

# Print the result
grouped_df


Grouped DataFrame with Total Ingredients Quantity:


,pizza_ingredients,total_qty_ingrams
0,?duja Salami,700.0
1,Alfredo Sauce,175.0
2,Anchovies,1005.0
3,Artichokes,2330.0
4,Arugula,220.0
...,...,...
57,Sun-dried Tomatoes,335.0
58,Thai Sweet Chilli Sauce,1090.0
59,Thyme,25.0
60,Tomatoes,11860.0
